In [1]:
from math import sqrt #크래머 v계수 계산용  
from scipy.stats import chi2_contingency #그래머 v계수 계산용  

In [4]:
import pandas as pd  
dat_df = pd.read_csv('./BehavioralDataAnalysis-master/Chapter 4 - Building Causal Diagrams From Scratch/chap4-hotel_booking_case_study.csv')
dat_df.head()

,NRDeposit,IsCanceled,DistributionChannel,MarketSegment,CustomerType,Children,ADR,PreviousCancellations,IsRepeatedGuest,Country,Quarter,Year
0,0,0,Direct,Direct,Transient,0,0.0,0,0,PRT,Q3,2015
1,0,0,Direct,Direct,Transient,0,0.0,0,0,PRT,Q3,2015
2,0,0,Direct,Direct,Transient,0,75.0,0,0,GBR,Q3,2015
3,0,0,Corporate,Corporate,Transient,0,75.0,0,0,GBR,Q3,2015
4,0,0,TA/TO,Online TA,Transient,0,98.0,0,0,GBR,Q3,2015


In [ ]:
NRD: 환불 불가능한 보증금이 있는지 여부, 이진형(0 혹은 1)  
isCancled: 예약 취소 여부, 이진형(0 혹은 1) 
DistributionChannel: 예약경로, 범주형 변수(direct(개인), corporate(법인), TA/TO(여행사), Other(기타)) 
MarketSegment: 시장 세부 유형, 범주형 변수  
(direct(개인), corporate(법인), Online TA/TO(온라인 여행사), Offline TA/TO(오프라인 여행사), Groups(단체), Other(기타))
CutomerType: 고객유형, 범주형 변수 (Transient(단기), Transient-party(단기 다중예약), Contract(계약), Group(단체))
Children: 어린이 인원수, 정수형 변수 
ADR: 1일 평균 요금, 총 예약 금액/예약 일수, 숫자형 변수 
PreviousCancellation: 고객의 취소 이력 여부, 이진형 변수(0 혹은 1)
IsRepeatedGuest: 고객의 예약 이력 여부, 이진형 변수(0 혹은 1)
Country: 고객의 출신 국가, 범주형 변수 
Quarter: 예약분기, 범주형 변수 
Year: 예약 연도, 정수형 변수

In [ ]:
#보증금 유형이 예약 취소율에 영향을 줄까?  
#환불 불가능한 보증금 여부(Y/N) > 예약 취소 여부(Y/N)  

In [12]:
#보증금 유형별 기본 취소율 확인
#수와 비율 확인 

table_cnt = dat_df.groupby(['NRDeposit', 'IsCanceled']).\
agg(cnt = ('Country', lambda x: len(x)))
print(table_cnt) 

#apply(함수, axis=0 혹은 1) #0은 열로 열은 디폴트 값이다. 
#level은 인덱스의 깊이를 의미한다. 가장 왼쪽이 0
table_pct = table_cnt.groupby(level=0).apply(lambda x: x/float(x.sum()))
print(table_pct)  

                        cnt
NRDeposit IsCanceled       
0         0           63316
          1           23042
1         0              55
          1             982
                           cnt
NRDeposit IsCanceled          
0         0           0.733180
          1           0.266820
1         0           0.053038
          1           0.946962


In [ ]:
#결과확인
보증금이 없는 예약이 8만6천건 이상으로 보증금이 없는 1000건 가량보다 높았고 
보증금 없는 취소율은 26.6퍼센트가 나왔다. 
반면 보증금이 있는 경우 예약 취소율은 95% 가량으로 매우 높았다. 
따라서 보증금 여부와 예약 취소율은 강한 상관관계를 보이는 것을 알 수 있다. 

그렇다면 보증금 제도를 없애면 취소율이 무조건 감소할까? No
실제로는 호텔이 취소 가능성이 높은 예약에 환불 불가 보증금을 거는 경우가 많고  
이 때문에 허위 상관관계가 발생했을 가능성이 높다.  

#교란 변수가 있는 인과관계 다이어그램
            선형적 취소 위험도 
          v                   v 
환불 불가능한 보증금 여부 > 예약 취소 여부   

현재의 다이어그램은 정보에 입각한 행동 가설을 나타낸다. 
환불이 불가능한 보증금이 취소율을 높이는 것처럼 보이지만 이 관계는 결정해야 하는  
요소에 의해 교란된 것일 수도 있다. 

인과관계 다이어그램에 대한 최소한의 지식이 있는 경우
휴가철 예약건에 환불이 불가능한 보증금을 거는건 
날씨 때문에 예약을 취소하는 경우가 많다는 것을 아는 것이다. 


In [ ]:
#잠깐########################################################################

In [39]:
dat_df.groupby(['NRDeposit', 'IsCanceled'])[['Country']].count()
#어째서인지 수가 안맞는다...차후 왜 그런지 원인규명 필요  

Country
NRDeposit IsCanceled         
0         0             62898
          1             23009
1         0                55
          1               981

In [34]:
#agg 메서드는 apply 메서드와 비슷하나 한번의 메서드에 여러 함수를 동시에 입력하고
#결과를 출력할 수 있다는 차이점이 있다.

#dat_df.groupby(['NRDeposit', 'IsCanceled']).describe()
dat_df.groupby(['NRDeposit', 'IsCanceled'])[['Country']].describe()

Country                   
                       count unique  top   freq
NRDeposit IsCanceled                           
0         0            62898    165  PRT  17633
          1            23009    127  PRT   8837
1         0               55      9  PRT     29
          1              981      9  PRT    953

In [11]:
dat_df.groupby(['NRDeposit', 'IsCanceled']).agg('Country').describe()

count unique  top   freq
NRDeposit IsCanceled                          
0         0           62898    165  PRT  17633
          1           23009    127  PRT   8837
1         0              55      9  PRT     29
          1             981      9  PRT    953

In [56]:
dat_df.groupby(['NRDeposit', 'IsCanceled']).agg(cnt = ('Country', lambda x: len(x)))

cnt
NRDeposit IsCanceled       
0         0           63316
          1           23042
1         0              55
          1             982

In [ ]:
#잠깐########################################################################